In [46]:
from __future__ import print_function
from __future__ import division
from gensim.models import Doc2Vec, Word2Vec
import lib.text_extraction as te
import lib.embedding as em
import lib.words as wd
import pandas as pd
import numpy as np
import operator
import codecs
import math
import json
import csv
import os
import re

In [2]:
test_dir = '../extraction/esempi_descrizioni/'

def list_dir(d):
    return [os.path.join(test_dir, f) for f in os.listdir(d)]

filenames = os.listdir(test_dir)
full_filenames = list_dir(test_dir)

txts = [te.extract_text(f, do_ocr=True) for f in full_filenames]


../extraction/esempi_descrizioni/5115873790001.pdf Estrazione testo con Vision API.
../extraction/esempi_descrizioni/02628680346_2081942240001.pdf Estrazione testo con Vision API.


In [10]:
for i, txt in zip(filenames, txts):
    print('Documento ', i)
    print(txt[:1000])

Documento  5120119780001.pdf
^ƚƵĚŝŽEŽƚĂƌŝůĞDŽƌŽŶĞ͘
ϭϬϭϮϭdŽƌŝŶŽͲǀŝĂDĞƌĐĂŶƚŝŶŝϱ
dĞůнϯϵϬϭϭϱϲϮϮϱϮϮͲ&ĂǆнϯϵϬϭϭϱϲϭϮϮϳϭ
ƐŽĐŝĞƚĂƌŝŽΛƐƚƵĚŝŽŵŽƌŽŶĞ͘ŝƚͲǁǁǁ͘ƐƚƵĚŝŽŵŽƌŽŶĞ͘ŝƚ

ĚůĐ

Repertorio n. 72.928

Raccolta n. 10.320

Costituzione della "NEXITY TRENTADUE S.r.l.".

Repubblica Italiana
Il ventuno giugno duemiladiciassette,

in Torino, nel mio Studio in via Mercantini n. 5.
Avanti me, avv. Francesco PENE VIDARI,
Notaio iscritto al Collegio Notarile dei Distretti
Riuniti di Torino e Pinerolo, con residenza in Torino, è personalmente comparso il signor
Christian BRAGAGNOLO, nato a San Remo (IM) il 6 luglio 1977, domiciliato a Torino, corso Galileo Ferraris n. 110,
nella sua qualità di procuratore della
"NEXITY HOLDING ITALIA S.r.l.", con sede in Tori-

no, corso Galileo Ferraris n. 110, col capitale sociale
delle

di

euro

30.010.000,

imprese

-

ufficio

iscritta
di

nel

Torino

regist
Documento  5114243810002-2.pdf
allegato "A" rep. n.28012 e racc. n.13815
STATUTO
Articolo 1 - 

In [21]:
extracted_txts_dir = '../extraction/esempi_extracted/'

for fn, txt in zip(filenames, txts):
    utxt = txt.decode('utf-8') if type(txt) == str else txt
    with codecs.open(os.path.join(extracted_txts_dir, fn[:-3])+'txt', 'w', encoding='utf-8') as o:
        o.write(utxt)


In [22]:
sentences_dir = '../extraction/esempi_sentences/'

sentences = [wd.sentences_doc(txt) for txt in txts]

for fn, txt in zip(filenames, sentences):
    with codecs.open(os.path.join(sentences_dir, fn[:-3])+'txt', 'w', encoding='utf-8') as o:
        o.write('\n'.join(s.replace('\n',' ').strip() for s in txt if s.strip()))

### Sentencing test

In [11]:
sentenced_txts = [wd.sentences_doc(txt) for txt in txts]

In [17]:
def sentence_label_csv_empty(filenames, sentenced_txts, csv_out, sep=u'\t', min_len=10):
    csv_out.write(sep.join([u'filename',u'sent_index',u'sentence',u'label']) + u'\n')
    for f, sents in zip(filenames, sentenced_txts):
        for i, sent in enumerate(sents):
            clean_sent = sent.replace(sep,u'').replace(u'\n',u' ').strip()
            if len(clean_sent)>=min_len:
                csv_out.write(sep.join([u'{}',u'{}',u'{}',u'non_rilevante']).format(f,i,clean_sent)+u'\n')
    

In [18]:
csv_filename = '../extraction/sentence_labels.csv'
with codecs.open(csv_filename, 'w', encoding='utf-8') as csv_out:
    sentence_label_csv_empty(os.listdir(test_dir), sentenced_txts, csv_out)

In [19]:
df = pd.read_csv(csv_filename, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE)

In [20]:
df

,filename,sent_index,sentence,label
0,5120119780001.pdf,0,^ E D d D d & Repertorio n 72928 Raccolta n 10...,non_rilevante
1,5120119780001.pdf,1,Repubblica Italiana Il ventuno giugno duemilad...,non_rilevante
2,5120119780001.pdf,2,"Detto signor comparente, della identità person...",non_rilevante
3,5120119780001.pdf,3,- 2 Sede La società ha sede nel Comune di Torino,non_rilevante
4,5120119780001.pdf,4,L'indirizzo della stessa è in corso Galileo Fe...,non_rilevante
5,5120119780001.pdf,5,"- l'acquisto, la vendita, la permuta e la gest...",non_rilevante
6,5120119780001.pdf,6,"- la lottizzazione di aree residenziali, indus...",non_rilevante
7,5120119780001.pdf,7,- la formazione di comparti secondo le normati...,non_rilevante
8,5120119780001.pdf,8,la partecipazione alla costituzione di consorz...,non_rilevante
9,5120119780001.pdf,9,la stipula di convenzioni ed atti d'obbligo pe...,non_rilevante


In [23]:
labeled_csv_filename = '../extraction/sentence_manual_labels.csv'

ldf = pd.read_csv(labeled_csv_filename, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE)

In [26]:
poteri = ldf.loc[ldf['label'] == 'poteri']['sentence']

In [32]:
def splitted_words_utf8(txt):
    return re.sub(ur'[^\w]',' ',txt, flags=re.UNICODE).split()


In [40]:
split_poteri = [splitted_words_utf8(s) for s in poteri]

In [45]:
def word_counts(sentences):
    d = dict()
    for sentence in sentences:
        for word in sentence:
            if not word in d:
                d[word] = 1
            else:
                d[word] += 1
    return d

def first_n_words(sentences, n):
    wc = word_counts(sentences)
    sorted_wc = sorted(wc.items(), key=operator.itemgetter(1))
    return list(reversed([x for x in sorted_wc[-n:]]))


first_n_words(split_poteri, 50)

[(u'di', 38),
 (u'poteri', 26),
 (u'e', 25),
 (u'per', 24),
 (u'della', 21),
 (u'dei', 21),
 (u'in', 20),
 (u'i', 17),
 (u'la', 17),
 (u'o', 15),
 (u'il', 14),
 (u'con', 12),
 (u'del', 12),
 (u'amministrazione', 12),
 (u'straordinaria', 11),
 (u'societ\xe0', 11),
 (u'che', 11),
 (u'pi\xf9', 10),
 (u'tutti', 10),
 (u'ordinaria', 10),
 (u'ad', 9),
 (u'organo', 9),
 (u'suoi', 9),
 (u'legge', 9),
 (u'amministrativo', 9),
 (u'presente', 9),
 (u'atto', 8),
 (u'soci', 8),
 (u'l', 8),
 (u'ed', 8),
 (u'ai', 7),
 (u'delle', 7),
 (u'componenti', 6),
 (u'\xe8', 6),
 (u'pu\xf2', 6),
 (u'L', 6),
 (u'le', 6),
 (u'gestione', 6),
 (u'dell', 6),
 (u'al', 5),
 (u'atti', 5),
 (u'quanto', 5),
 (u'facolt\xe0', 5),
 (u'Il', 5),
 (u'ampi', 5),
 (u'delegare', 5),
 (u'parte', 5),
 (u'uno', 5),
 (u'limiti', 5),
 (u'eccezione', 5)]

## Tf Idf


In [47]:
# Very inefficient...

def tf(word, sentence):
    return sum(1 for w in sentence if w == word)/len(sentence)

def idf(word, sentences):
    D = len(sentences)
    den = sum(1 for sentence in sentences if word in sentence)
    return math.log(D/den)

def tf_idf(word, sentence, sentences):
    return tf(word, sentence)*idf(word, sentences)

In [54]:
split_sentences = [splitted_words_utf8(s) for s in ldf['sentence']]

In [59]:
split_poteri[0]

[u'In',
 u'mancanza',
 u'di',
 u'qualsiasi',
 u'precisazione',
 u'nell',
 u'atto',
 u'di',
 u'nomina',
 u'in',
 u'ordine',
 u'alle',
 u'modalit\xe0',
 u'di',
 u'esercizio',
 u'dei',
 u'poteri',
 u'di',
 u'amministrazione',
 u'detti',
 u'poteri',
 u'si',
 u'intendono',
 u'attribuiti',
 u'agli',
 u'ammini',
 u'stratori',
 u'disgiuntamente',
 u'tra',
 u'loro']

In [60]:
tf_idf('amministrazione', split_poteri[0], split_sentences)

0.08679417153318603

In [62]:
for s in split_poteri:
    print(tf_idf('poteri', s, split_sentences))


0.216994939354
0.197268126686
0.0793883924466
0.0140905804775
0.130196963612
0.0
0.162746204516
0.0879709213598
0.162746204516
0.0986340633428
0.0
0.141518438709
0.120552744086
0.057103931409
0.1049975513
0.0439854606799
0.0625946940445
0.0678109185482
0.0929978311518
0.0813731022578
0.0524987756502
0.197268126686
0.180829116128
0.171311794227
0.250378776178


## Embeddings test

In [10]:
txts_tokenized = [wd.tokenize_doc(txt) for txt in txts]

In [11]:
reduced_dictionary_filename = 'first_5000_words.json'
with open(reduced_dictionary_filename) as f:
    reduced_dictionary = set(json.load(f))

gensim_model_filename = 'models/gensim_model_5000.d2v'
gensim_model = Doc2Vec.load(gensim_model_filename)

In [12]:
embeddings = [em.embed_document(gensim_model, txt, reduced_dictionary) for txt in txts_tokenized]

In [13]:
for i, s in enumerate(txts_tokenized[-1]):
    print(i, s)

0 [u'epertorio', u'umero', u'NUM', u'ascicolo', u'umero', u'NUM', u'anno', u'duemiladiciassette', u'alli', u'ventotto', u'mese', u'giugno', u'mperia', u'orto', u'aurizio', u'ia', u'elice', u'ascione', u'numero', u'NUM', u'piano', u'primo']
1 [u'nnanzi', u'me', u'ottor', u'otaio', u'mperia', u'iscritto', u'presso', u'ollegio', u'otarile', u'istretti', u'iuniti', u'mperia', u'anremo', u'personalmente', u'comparsi', u'signori', u'nato', u'rtova', u'urchia', u'NUM', u'aprile', u'NUM', u'residente', u'mperia', u'frazione', u'antalupo', u'trada', u'orsa', u'n', u'NUM', u'cittadino', u'turco', u'codice', u'fiscale', u'NUM', u'NUM', u'NUM', u'titolare', u'permesso', u'soggiorno', u'durata', u'illimitata', u'numero', u'NUM', u'rilasciato', u'mperia', u'NUM', u'ottobre', u'NUM', u'dichiara', u'ben', u'conoscere', u'lingua', u'italiana']
2 [u'nato', u'okat', u'urchia', u'3', u'settembre', u'NUM', u'residente', u'mperia', u'ia']
3 [u'irenti', u'n', u'NUM', u'cittadino', u'turco', u'codice', u'fisc

In [14]:
def cosine_similarity(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))


In [15]:
cosine_similarity(embeddings[-1][20],embeddings[-1][-1])

0.45552456

In [16]:
sentences_tokenized = [sent for doc in txts_tokenized for sent in doc] #i should reduce the dictionary also

In [17]:
w2v = Word2Vec(sentences_tokenized, size=100, window=5, min_count=5, workers=4)

In [18]:
w2v.wv['costitutivo']

array([ 0.19403476,  0.29544842,  0.19333051,  0.07171968, -0.16629422,
       -0.27712828,  0.10267814, -0.05072037,  0.00967973, -0.08712386,
       -0.20192116, -0.06190296,  0.02626663,  0.0765117 ,  0.28788161,
       -0.19467854,  0.11269861, -0.26862299, -0.04416437, -0.25605428,
        0.00450306, -0.02204379, -0.06437516,  0.06308224, -0.03666903,
       -0.39022329, -0.08860961,  0.32243007,  0.20351775,  0.30535963,
       -0.07628474,  0.15784067,  0.07071318, -0.15113319, -0.04585525,
       -0.31896675, -0.21602617,  0.19662151, -0.28631762,  0.10489404,
       -0.11170984, -0.16622221,  0.02484445,  0.30350059, -0.16141514,
       -0.04411188, -0.07407032,  0.00376601, -0.23364276,  0.13584408,
        0.19938666, -0.39598995, -0.10570914,  0.32415739, -0.06165232,
       -0.0428248 ,  0.09726623, -0.29649934,  0.40978107,  0.15958688,
        0.10775582, -0.1612976 , -0.20271511,  0.08806681, -0.1895763 ,
       -0.00510498, -0.02566624, -0.29317275, -0.15035689,  0.17

In [19]:
cosine_similarity(w2v.wv['atto'], w2v.wv['costitutivo'])

0.99939936

In [20]:
w2v.most_similar('poteri')

[(u'qualsiasi', 0.9999268054962158),
 (u'nonch\xe9', 0.9999196529388428),
 (u'carica', 0.9999138712882996),
 (u'comunque', 0.9999135732650757),
 (u'assemblea', 0.999913215637207),
 (u'ociet\xe0', 0.9999076724052429),
 (u'quali', 0.999907374382019),
 (u'senza', 0.9999068379402161),
 (u'atti', 0.9999067187309265),
 (u'societ\xe0', 0.9999057054519653)]

In [21]:
w2v.most_similar(positive=['atto','costitutivo'])

[(u'documento', 0.999783456325531),
 (u'presente', 0.9997665286064148),
 (u'statuto', 0.9997603893280029),
 (u'originale', 0.999711275100708),
 (u'sotto', 0.9996922016143799),
 (u'otaio', 0.9996887445449829),
 (u'norme', 0.9996862411499023),
 (u'legge', 0.9996805191040039),
 (u'seguenti', 0.9996784925460815),
 (u'via', 0.9996722340583801)]